This tutorial demonstrates how the composition of a ternary heterostructured semiconductor can be found using an internal reference area for two of the elements within the scanned area (when available). 

The example data was acquired from a 111 cross-section of a GaAs/AlGaAs core-shell nanowire, where the Al content in the shell is non-uniform. The area of interest that is scanned covers two of the six nanowire shells as well as part of the nanowire core.

Import necessary libraries:

In [ ]:
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
from hyperspy import utils

Load and plot dataset:

In [ ]:
dataset = 'NW_cross_section_EDS.hdf5'
s = hs.load(dataset)
s.plot()

Sum one axis to create linescan and increase X-ray counts:

Note: It is also possible to perform a denoising of the data, e.g. by principal component analysis, but is not included in this tutorial. For more detail on default EDX processing in  HyperSpy see demo’s.

https://github.com/hyperspy/hyperspy-demos/blob/master/electron_microscopy/EDS/SEM_EDS_4D_visualisation.ipynb

In [ ]:
width = s.axes_manager[1].size
s = s.sum(axis=1)
s.plot()

Define necessary parameters:

In [ ]:
constant_element = 'As' # The element for which the composition is constant over the scanned area
unknown_zeta = 'Al' # The element for which there is no internal reference
probe_current = 0.692 # nA (Can be arbitrary)
acquisition_time = 0.1 # s (Can be arbitrary)
elements = ['Al','As','Ga'] # The elements of the area of interest
lines = ['Al_Ka','As_Ka','Ga_Ka'] # The X-ray lines used for quantification

comp_ref = hs.material.atomic_to_weight([0.5, 0.5], ("As", "Ga"))/100 # Composition of ref. area in wt.%
density_ref = hs.material.density_of_mixture(comp_ref, ("As","Ga")) * 1e3 # Density of ref. in kg/m^2
lines_ref = ["As_Ka","Ga_Ka"] # The X-ray lines used in the reference area
thickness = 45 # nm (Can be arbitrary)

Define the reference area, sum the counts, set the background windows and find the total counts:

In [ ]:
ref_area = s.inav[123:302] # The pixels in the data set to be used as a reference area.
length = ref_area.axes_manager[0].size
ref_area = ref_area.sum()
ref_area.add_lines(lines_ref)
bw_As_ref = ref_area.estimate_background_windows(line_width=[3,3], windows_width=1, xray_lines=['As_Ka'])
bw_Ga_ref = ref_area.estimate_background_windows(line_width=[3,3], windows_width=1, xray_lines=['Ga_Ka'])
BW_ref = np.concatenate((bw_As_ref, bw_Ga_ref)) # must be in alphabetical order
ref_area.plot(background_windows=BW_ref)
ref_area_ints = ref_area.get_lines_intensity(lines = lines_ref, 
                                             background_windows=BW_ref, 
                                             integrations_windows=1.2)

Set background windows and find the X-ray counts in the area of interest:

In [ ]:
s.add_lines(lines)
bw_1 = s.estimate_background_windows(line_width=[8,2], windows_width=1, xray_lines=['Al_Ka'])
bw_2 = s.estimate_background_windows(line_width=[3,3], windows_width=1, xray_lines=['As_Ka'])
bw_3 = s.estimate_background_windows(line_width=[3,3], windows_width=1, xray_lines=['Ga_Ka'])
BW = np.concatenate((bw_1, bw_2, bw_3)) # must be in alphabetical order
s.plot(background_windows=BW)
s_ints = s.get_lines_intensity(background_windows=BW, integrations_windows=1.2)

Define functions needed:

In [ ]:
def determine_zeta_factor(intensities, composition, dose, mass_thickness):
    """
    Determine zeta-factors from the reference area.

    Parameters
    ----------
    intensities: numpy.array
        The intensities for each X-ray line in the reference. The first axis should be the element axis.
    composition: list of float
        Composition of the elements given in wt.% in the same order as intensities.
    dose: float
        Total electron dose given by i*t*N, where i is the beam_current, t is the acquisition time,
        and N the number of electrons per unit electric charge (1/e).

    Returns:
    -------
    The zeta factors of the reference area.
    """
        
    zfactors = np.zeros_like(intensities, dtype='float')
    
    for i, (intensity, comp) in enumerate(zip(intensities, composition)):
        zfactors[i] = (dose * mass_thickness * comp) / intensity

    return zfactors

def calculate_unknown_zeta(elements, zfactors, intensities, const_el):
    '''
    Calculates the unknown zeta factor in a system where the other zeta factors are known, and one of the
    elements is known to be constant with respect to the concentration of the element with unknown zeta 
    factor. A good example is ternary heterostructured semiconductors where one of the elements in known 
    to be 50 at.% independently of the other two elements.
    
    Parameters:
    ------------
    elements: list
        The elements present in the sample, in the same order as intensities (alphabetical).
    zfactors: list of ints
        The zeta factors of the system. The unknown zeta factor is set to 0, same order as intensities.
    intensities: numpy.array
        The intensities for each X-ray line. The first axis should be the element axis.
    const_el: str
        The element with the constant concentration.
    
    Returns:
    --------
    The unknown zeta factor of the element that gives the most constant concentration of the element that 
    is known to have a constant concentration.
    '''

    unknown_zeta = zfactors.index(0)
    constant_element = elements.index(const_el)
    flatness = 0
    
    while(True):
        
        zI = []
        for i in range(len(zfactors)):
            zI.append(zfactors[i]*intensities[i].data)
        
        comp_wt = []
        for i in range(len(zfactors)):
            comp_wt.append(zI[i]/sum(zI))

        comp_at = hs.material.weight_to_atomic([comp_wt[0], comp_wt[1], comp_wt[2]], (elements))/100
    
        if (np.max(comp_at[constant_element])-np.min(comp_at[constant_element])) < flatness:
            print("The zeta factor for " + elements[unknown_zeta] + " was determined to be " + 
                  str(zfactors[unknown_zeta]) + " with a flatness criterion of " 
                  + str(round(flatness*100,2)) + " at.%.")
            break
        elif zfactors[unknown_zeta] > (10*(zfactors[constant_element])):
            flatness += 0.001
            zfactors[unknown_zeta] = 0
    
        zfactors[unknown_zeta] += 1
    
    return zfactors[unknown_zeta], flatness

def find_zeta_from_reference(r, intensities_ref, acquisition_time, probe_current, thickness, density_ref, 
                             composition_ref):
    '''
    Finds the zeta factors of the reference area.
    Parameters:
    -----------
    r: The EDX signal of the reference area.
    intensities_ref: numpy.array
        The intensities for each X-ray line in the reference. The first axis should be the element axis.
    acquisition time: float
        The dwell time of each pixel (can be arbitrary)
    probe_current: float
        The probe current during the aqcuisition (can be arbitrary)
    thickness: float
        The specimen thickness of the refrence area in nm (can be arbitrary)
    density_ref: float
        The density of the reference area.
    composition_ref: list of float
        Composition of the elements in the reference given in wt.% in the same order as intensities.
        
    Returns:
    ---------
    The zeta factors of the reference area.
    '''
    
    r.set_microscope_parameters(live_time = acquisition_time*width*length, beam_current=probe_current)
    intensities_ref = utils.stack(intensities_ref)
    zeta_factors = determine_zeta_factor(intensities_ref, 
                                         composition_ref, 
                                         r._get_dose("zeta"), 
                                         thickness*1e-9*density_ref)
    
    return zeta_factors

def find_unknown_zeta(intensities, elements, constant_el, known_zeta, lines_ref):
    '''
    Calculates the uknown zeta factor based on the zeta factors found in the reference area and that one
    element has constant composition.
    Paramenters:
    --------------
    intensities: numpy.array
        The intensities of each X-ray line in the aera of interest. The first axis should be the element axis.
    elements: list of str
        The elements to be quantified in the area of interest, in the same order as intensities.
    known_zeta: list of float
        The zeta factors found from the reference area.
    lines_ref: List of str
        The X-ray lines for the zeta factors of the reference area.
     
    Returns:
    ---------
    The zeta factors for the three elements.
    '''
    
    zfactors = [0., 0., 0.]
    zfactors[lines.index(lines_ref[0])] = known_zeta[0]
    zfactors[lines.index(lines_ref[1])] = known_zeta[1]
    zeta_unknown = calculate_unknown_zeta(elements, zfactors, intensities, const_el=constant_element)
    
    return zfactors

def composition_thickness(s, acquisition_time, probe_current, intensities, zeta_factors):
    '''
    Calculates the composition and thickness of the area of interest.
    Parameters:
    ------------
    s: The EDX data to be quantified
    aquisition_time: float
        The dwell time of each pixel (can be arbitrary, but same as when calc. zeta factors)
    probe_current: float
        The probe current during the aqcuisition (can be arbitrary, but same as when calc. zeta factors)
    zeta_factors: list of float
        The zeta factors for the X-ray lines of the elements to be quantified.
        
    Returns:
    -----------
    The composition and thickness of the area of interest.
    '''
    
    s.set_microscope_parameters(live_time=acquisition_time*width, beam_current=probe_current)
    composition, pt = s.quantification(method='zeta', 
                                       intensities=intensities, 
                                       factors=zeta_factors, 
                                       composition_units='atomic')
    p = hs.material.density_of_mixture(hs.material.atomic_to_weight(composition)) * 1e3
    thickness = pt*1e9/p
    
    return composition, thickness

Find the zeta factors of the reference area:

In [ ]:
zeta_from_reference = find_zeta_from_reference(ref_area,
                                               ref_area_ints,
                                               acquisition_time,
                                               probe_current,
                                               thickness,
                                               density_ref,
                                               comp_ref)

Calculate the unknown zeta factor:

In [ ]:
known_zeta = []
for i in range(len(zeta_from_reference)):
     known_zeta.append(zeta_from_reference[i][0])
        
zeta_factors = find_unknown_zeta(s_ints,
                                 elements,
                                 constant_element,
                                 known_zeta,
                                 lines_ref)

Calculate the composition and thickness based on the found zeta-factors:

In [ ]:
composition, thickness = composition_thickness(s,
                                             acquisition_time,
                                             probe_current,
                                             s_ints,
                                             zeta_factors)

Plot the composition profile:

In [ ]:
f = plt.figure(tight_layout=True, figsize=(8,4))

x = composition[0].axes_manager[0].axis

plt.plot(x, composition[0].data, 'b', label='Al')
plt.plot(x, composition[1].data, 'r', label='As')
plt.plot(x, composition[2].data, 'g', label='Ga')

plt.xlabel('nm')
plt.ylabel('at.%')
plt.legend(loc='lower right')
plt.xlim([0,x[-1]])
plt.ylim([0, 55])

plt.show()

Plot the thickness profile:

In [ ]:
plt.figure(tight_layout=True, figsize=(8,4))
title = 'Thickness' 
x = composition[0].axes_manager[0].axis
plt.plot(x, thickness.data, 'k')
plt.title(title)
plt.xlabel('nm')
plt.ylabel('Thickness [nm]')
plt.xlim([0, x[-1]])